In [1]:
import os
from openai import OpenAI, pydantic_function_tool
from dotenv import load_dotenv
import rich
import requests
import json
from pydantic import BaseModel, Field

In [2]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI()

Calling the same function multiple times when response from Chat API and Responses API requires to call function multiple times

The Pydantic-generated function structure is acceptable in OpenAI's Chat API, but the Responses API requires a slightly different structure

In [3]:
class GetWeather(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")
    
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(f"get_weather function called to get weather for latitude = {latitude}, longitude = {longitude}")
    print(f"And result is  = {data['current']['temperature_2m']}")
    return data['current']['temperature_2m']

rich.print(pydantic_function_tool(GetWeather))

{
    'type': 'function',
    'function': {
        'name': 'GetWeather',
        'strict': True,
        'parameters': {
            'properties': {
                'latitude': {'description': 'Latitude of the location', 'title': 'Latitude', 'type': 'number'},
                'longitude': {'description': 'Longitude of the location', 'title': 'Longitude', 'type': 'number'}
            },
            'required': ['latitude', 'longitude'],
            'title': 'GetWeather',
            'type': 'object',
            'additionalProperties': False
        }
    }
}

# Chat Completion API

https://platform.openai.com/docs/guides/function-calling?api-mode=chat

In [4]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city. Response should be courteous and professional."},
    # {"role": "user", "content": "What's the weather like in Karachi and Lahore?"}
    # {"role": "user", "content": "NYC"}
    {"role": "user", "content": "Berlin and Paris"}
]
tools = [pydantic_function_tool(GetWeather)] # Except for this line, everything else is same as previous example
response = openai.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools = tools
)

rich.print(response.choices[0])
print("Finish Reason = ", response.choices[0].finish_reason)
rich.print(response.choices[0].message.tool_calls) # Chat API returns tool_calls for all expected tools in single response
print("Number of tool calls: ",len(response.choices[0].message.tool_calls))

Choice(
    finish_reason='tool_calls',
    index=0,
    logprobs=None,
    message=ChatCompletionMessage(
        content=None,
        refusal=None,
        role='assistant',
        annotations=[],
        audio=None,
        function_call=None,
        tool_calls=[
            ChatCompletionMessageToolCall(
                id='call_MDu00ueSP63dtNxqSAfb8JOE',
                function=Function(arguments='{"latitude": 52.52, "longitude": 13.405}', name='GetWeather'),
                type='function'
            ),
            ChatCompletionMessageToolCall(
                id='call_GyAhWoY2TwdOH49JkvETOikW',
                function=Function(arguments='{"latitude": 48.8566, "longitude": 2.3522}', name='GetWeather'),
                type='function'
            )
        ]
    )
)

Finish Reason =  tool_calls


[
    ChatCompletionMessageToolCall(
        id='call_MDu00ueSP63dtNxqSAfb8JOE',
        function=Function(arguments='{"latitude": 52.52, "longitude": 13.405}', name='GetWeather'),
        type='function'
    ),
    ChatCompletionMessageToolCall(
        id='call_GyAhWoY2TwdOH49JkvETOikW',
        function=Function(arguments='{"latitude": 48.8566, "longitude": 2.3522}', name='GetWeather'),
        type='function'
    )
]

Number of tool calls:  2


In [5]:
# We will loop through the tool_calls, invoke the corresponding functions, and update the response. 
# Plus, we will append a new message for each function call to promptMessages.
def handle_tool_call(promptMessages, responseMessage):
    for tool_call in responseMessage.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        weather = get_weather(**arguments)
        new_message = {
            "role": "tool",
            "content": str(weather),
            "tool_call_id": tool_call.id
        }
        promptMessages.append(new_message)

In [6]:
if response.choices[0].finish_reason == "tool_calls": # Check if finish_reason is tool_calls
    messages.append(response.choices[0].message) # Important: we will append the previous message (response.choices[0].message)
    handle_tool_call(messages, response.choices[0].message)
    response2 = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print("Model Response2 = ",response2.choices[0].message.content)
    print("Finish Reason = ",response2.choices[0].finish_reason)

get_weather function called to get weather for latitude = 52.52, longitude = 13.405
And result is  = 6.0
get_weather function called to get weather for latitude = 48.8566, longitude = 2.3522
And result is  = 11.6
Model Response2 =  The current weather in Berlin is approximately 6.0°C, while in Paris, it is around 11.6°C. If you need more specific details or forecasts, feel free to ask!
Finish Reason =  stop


# Responses API

https://platform.openai.com/docs/guides/function-calling?api-mode=responses

In [7]:
rich.print(pydantic_function_tool(GetWeather)) # This same as above

{
    'type': 'function',
    'function': {
        'name': 'GetWeather',
        'strict': True,
        'parameters': {
            'properties': {
                'latitude': {'description': 'Latitude of the location', 'title': 'Latitude', 'type': 'number'},
                'longitude': {'description': 'Longitude of the location', 'title': 'Longitude', 'type': 'number'}
            },
            'required': ['latitude', 'longitude'],
            'title': 'GetWeather',
            'type': 'object',
            'additionalProperties': False
        }
    }
}

To use pydantic-generated function, we need to use openai.responses.parse() function call

https://github.com/openai/openai-python/blob/main/examples/responses/structured_outputs_tools.py

Somehow Responses API is not calling weather function for all cities in one go

In [8]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city. Response should be courteous and professional."},
    {"role": "user", "content": "What's the weather like in Karachi and Lahore?"}
    # {"role": "user", "content": "Berlin and Paris"}
]
# messages=[
#     {"role": "developer", "content": "You are a helpful assistant and provide update on weather in all the cities as asked by user. Response should be courteous and professional."},
#     {"role": "user", "content": "What's the weather in following cities Karachi and Lahore?"}
# ]
tools = [pydantic_function_tool(GetWeather)]

# Somehow Responses API is not calling weather function for all cities in one go
response = openai.responses.parse(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output) 
# In this case Responses API calling tools one by one. We need to send the response for the first 
# tool call to the API, after which the Responses API will send the next tool call.

print("Number of tool calls: ",len(response.output))
# rich.print(response)

Status =  completed



[
    ParsedResponseFunctionToolCall(
        id='fc_67da99abfe9c8192b6cd158d94f40dbc05ebc1cb468816dd',
        arguments='{"latitude":24.8607,"longitude":67.0011}',
        call_id='call_4dbmY6axllha5Kh1sNy3AwNP',
        name='GetWeather',
        type='function_call',
        status='completed',
        parsed_arguments=GetWeather(latitude=24.8607, longitude=67.0011)
    )
]

Number of tool calls:  1


Using example of different function to test multiple calls of same function in single response

In [9]:
class SendEmail(BaseModel):
    to: str = Field(..., description="Email address of the recipient")
    subject: str = Field(..., description="Subject of the email")
    body: str = Field(..., description="Body of the email")


def send_email(to, subject, body):
    print(f"Tool send_email Sending email to {to} with subject {subject}")
    print(f"Body: {body}")
    print(f"Email Tool call completed")
    return "Email Sent"

### Using old way of sending history messages in every call

In [10]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant, you can send email about weather in a city. Email should be courteous and professional."},
    {"role": "user", "content": "send an email to first@gmail.com and second@gmail.com saying Hello"}
]
tools = [pydantic_function_tool(SendEmail)]

response = openai.responses.parse(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output) 

print("Number of tool calls: ",len(response.output))
# rich.print(response)

Status =  completed



[
    ParsedResponseFunctionToolCall(
        id='fc_67da99ca25a0819282cbaea5f547b2350bc10cd23bcaa286',
        arguments='{"to":"first@gmail.com","subject":"Greetings","body":"Hello"}',
        call_id='call_EuJdUma345l74eh3dS6MXaHT',
        name='SendEmail',
        type='function_call',
        status='completed',
        parsed_arguments=SendEmail(to='first@gmail.com', subject='Greetings', body='Hello')
    ),
    ParsedResponseFunctionToolCall(
        id='fc_67da99ca90c481928d8878616174bd080bc10cd23bcaa286',
        arguments='{"to":"second@gmail.com","subject":"Greetings","body":"Hello"}',
        call_id='call_rI47bHNoS02JpEf3Oqqz9KMr',
        name='SendEmail',
        type='function_call',
        status='completed',
        parsed_arguments=SendEmail(to='second@gmail.com', subject='Greetings', body='Hello')
    )
]

Number of tool calls:  2


In [11]:
# Call the relevant function and return the output
def handle_tool_call_responses(tool_call):
    result = send_email(tool_call.parsed_arguments.to, tool_call.parsed_arguments.subject,tool_call.parsed_arguments.body) 

    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    }
    return new_message

In [183]:
# Logic here is bit different then what we have done in Chat API
tool_call_results = []
for tool_call in response.output:
    if tool_call.type == "function_call": # check if the output's type is function_call
        tool_call_results.append(handle_tool_call_responses(tool_call))

for item in response.output:
    del item.parsed_arguments
    
messages.extend(response.output) # For multiple tool calls, we need to append the output of each tool call individually
messages.extend(tool_call_results) # Adding the result of tool calls individually
# rich.print(messages)
response2 = openai.responses.parse(model=MODEL, input=messages,tools = tools)
print("Model Response2 output_text = ",response2.output_text)
# rich.print("Model Response2 = ",response2)
print("Status = ",response2.status)

Tool send_email Sending email to first@gmail.com with subject Greeting
Body: Hello
Email Tool call completed
Tool send_email Sending email to second@gmail.com with subject Greeting
Body: Hello
Email Tool call completed
Model Response2 output_text =  I have sent a greeting email to both **first@gmail.com** and **second@gmail.com**. If you need anything else, feel free to ask!
Status =  completed


### Using new way of conversation state by sending perivous reponse id

In [12]:
# This section is same as above
messages=[
    {"role": "developer", "content": "You are a helpful assistant, you can send email about weather in a city. Email should be courteous and professional."},
    {"role": "user", "content": "send an email to first@gmail.com and second@gmail.com saying Hello"}
]
tools = [pydantic_function_tool(SendEmail)]

response = openai.responses.parse(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output) 

print("Number of tool calls: ",len(response.output))
# rich.print(response)

Status =  completed



[
    ParsedResponseFunctionToolCall(
        id='fc_67da9b665058819293161a159fa336f60ad527867692d709',
        arguments='{"to":"first@gmail.com","subject":"Greetings","body":"Hello!"}',
        call_id='call_Gb4xvbQTwZgNOrjeeJ16FHR2',
        name='SendEmail',
        type='function_call',
        status='completed',
        parsed_arguments=SendEmail(to='first@gmail.com', subject='Greetings', body='Hello!')
    ),
    ParsedResponseFunctionToolCall(
        id='fc_67da9b6693e88192aa33bcd3cbf5b4790ad527867692d709',
        arguments='{"to":"second@gmail.com","subject":"Greetings","body":"Hello!"}',
        call_id='call_0ej4c41zixxrsQtvVmRPkEHV',
        name='SendEmail',
        type='function_call',
        status='completed',
        parsed_arguments=SendEmail(to='second@gmail.com', subject='Greetings', body='Hello!')
    )
]

Number of tool calls:  2


In [13]:
# This section is same as above

# Call the relevant function and return the output
def handle_tool_call_responses(tool_call):
    result = send_email(tool_call.parsed_arguments.to, tool_call.parsed_arguments.subject,tool_call.parsed_arguments.body) 

    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    }
    return new_message

In [ ]:
# The only difference in this section is how messages are sent.

# Logic here is bit different then what we have done in Chat API
tool_call_results = []
for tool_call in response.output:
    if tool_call.type == "function_call": # check if the output's type is function_call
        tool_call_results.append(handle_tool_call_responses(tool_call))

# for item in response.output:   # Not needed now because we are using response.id
#     del item.parsed_arguments
    
# messages.extend(response.output) # Not needed now because we are using response.id
messages.extend(tool_call_results) # Adding the result of tool calls individually
# rich.print(messages)
response2 = openai.responses.parse(model=MODEL, input=messages,tools = tools, previous_response_id=response.id)
print("Model Response2 output_text = ",response2.output_text)
# rich.print("Model Response2 = ",response2)
print("Status = ",response2.status)

Tool send_email Sending email to first@gmail.com with subject Greetings
Body: Hello!
Email Tool call completed
Tool send_email Sending email to second@gmail.com with subject Greetings
Body: Hello!
Email Tool call completed
Model Response2 output_text =  I have sent an email saying "Hello!" to both first@gmail.com and second@gmail.com. If you need any further assistance or want to add more details, feel free to let me know!
Status =  completed
